In [3]:
# data handling
import pandas as pd 
import numpy as np 
from datetime import datetime
import scipy 
import math 

# visualizations
import matplotlib.pyplot as plt 
import seaborn as sns

# pickle 
import pickle 

# sklearn 
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, classification_report, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# pytorch 
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data 
from torch.autograd import Variable
import torch.optim as optim 

# optuna 
import optuna 

In [7]:
SEED = 1

# check for GPU 
DEVICE = 'cpu'
cuda = torch.cuda.is_available()
if cuda: 
    print('cuda available..')
    DEVICE = 'cuda'
    torch.cuda.manual_seed(SEED)
print(f'Device: {DEVICE}')

cuda available..
Device: cuda


# Data Notes
---
Files (by fishing gear types):  
1. Drifting longline vessels 
2. Fixed gear vessels 
3. Pole and line vessels 
4. Purse seines vessels
5. Trawler vessels 
6. Troller vessels 
7. Vessels with unknown gear types  
  
CSV Table Schema:  
- mmsi: anonymized vessel identifier
- timestamp: unix timestamp
- distance_from_shore: distance from shore in meters
- distance_from_port: distance from port in meters
- speed: vessel speed in knots
- course: vessel's course over ground (represented in degrees)
    - Some records will have course values of 511, which is larger than the maximum value of 360 degrees. That means the data is not available according to the [US Coast Guard Class A AIS Position Report Documentation](https://www.navcen.uscg.gov/?pageName=AISMessagesA).
- lat: latitude in decimal degrees
- lon: longitude in decimal degrees
- is_fishing: lable indicating fishing activity
    - 0 = not fishing
    - \> 0 = fishing; data values between 0 and 1 indicate the average score for the position if scored by multiple people
    - -1 = no data
- source: the training data batch; data was prepared by GFW, Dalhousie, and a crowd sourcing campaign (false positives are marked as false_positives)

In [9]:
# filepaths 
drifting_longlines_file = './data/drifting_longlines.csv'
fixed_gear_file = './data/fixed_gear.csv'
pole_and_line_file = './data/pole_and_line.csv'
purse_seines_file = './data/purse_seines.csv'
trawlers_file = './data/trawlers.csv'
trollers_file = './data/trollers.csv'
unknown_file = './data/unknown.csv'

# for mapping vessel paths later 
# ocean_map = gpd.read_file('./mapping/world_map/ne_50m_ocean.shp')
# south_atlantic = gpd.read_file('./mapping/south_atlantic/iho.shp')
# bornholm = gpd.read_file('./mapping/bornholm/europeislands.shp')
# arrecife = gpd.read_file('./mapping/north_atlantic/iho.shp')
# phillippine_sea = gpd.read_file('./mapping/phillippine_sea/iho.shp')
# north_pacific = gpd.read_file('./mapping/north_pacific/eez_iho.shp')
# crs = {'init': 'epsg:4326'}

# Exploratory Analysis / Initial Data Wrangling on Separated Datasets  
---  
  


## Drifting Longlines


In [27]:
# read in data from the drifting long line vessel file 
drifting_longlines_df = pd.read_csv(drifting_longlines_file)

# peak at the data
drifting_longlines_df.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.263956e+13,1.327137e+09,232994.281250,311748.65625,8.2,230.500000,14.865583,-26.853662,-1.0,dalhousie_longliner
1,1.263956e+13,1.327137e+09,233994.265625,312410.34375,7.3,238.399994,14.863870,-26.856800,-1.0,dalhousie_longliner
2,1.263956e+13,1.327137e+09,233994.265625,312410.34375,6.8,238.899994,14.861551,-26.860649,-1.0,dalhousie_longliner
3,1.263956e+13,1.327143e+09,233994.265625,315417.37500,6.9,251.800003,14.822686,-26.865898,-1.0,dalhousie_longliner
4,1.263956e+13,1.327143e+09,233996.390625,316172.56250,6.1,231.100006,14.821825,-26.867579,-1.0,dalhousie_longliner


In [28]:
# check data characteristics 
drifting_longlines_df.describe()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing
count,1.396873e+07,1.396873e+07,1.396873e+07,1.396873e+07,1.396863e+07,1.396863e+07,1.396873e+07,1.396873e+07,1.396873e+07
mean,1.293850e+14,1.434290e+09,5.845311e+05,7.897505e+05,5.464779e+00,1.814876e+02,-8.997629e+00,3.758693e+00,-9.743015e-01
std,7.887357e+13,3.984275e+07,5.420068e+05,6.915438e+05,4.043567e+00,1.050503e+02,2.439311e+01,1.095971e+02,2.119947e-01
min,5.601266e+12,1.325376e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.519017e+01,-1.800000e+02,-1.000000e+00
25%,6.260384e+13,1.410706e+09,1.019092e+05,2.130206e+05,2.100000e+00,9.070000e+01,-2.601550e+01,-8.808668e+01,-1.000000e+00
50%,1.184859e+14,1.447302e+09,4.576393e+05,6.375249e+05,5.500000e+00,1.811000e+02,-1.497954e+01,-1.716495e+00,-1.000000e+00
75%,1.980758e+14,1.466506e+09,9.603664e+05,1.210432e+06,8.500000e+00,2.711000e+02,4.485790e+00,1.009811e+02,-1.000000e+00
max,2.812058e+14,1.480032e+09,4.430996e+06,7.181037e+06,1.023000e+02,5.110000e+02,8.333266e+01,1.799938e+02,1.000000e+00


In [29]:
# data shape 
drifting_longlines_df.shape

(13968727, 10)

In [30]:
# drop rows without data for is_fishing 
drifting_longlines_df = drifting_longlines_df[drifting_longlines_df['is_fishing'] != -1]

# recheck data shape
drifting_longlines_df.shape

(219741, 10)

In [31]:
# check label distribution 
drifting_longlines_df['is_fishing'].value_counts()

1.000000    138163
0.000000     79574
0.666667      1076
0.333333       809
0.750000       110
0.250000         9
Name: is_fishing, dtype: int64

In [15]:
# recheck data characteristics after dropping rows missing is_fishing data 
drifting_longlines_df.describe()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing
count,1.383056e+07,1.383056e+07,1.383056e+07,1.383056e+07,1.383047e+07,1.383047e+07,1.383056e+07,1.383056e+07,1.383056e+07
mean,1.293147e+14,1.434526e+09,5.845252e+05,7.894688e+05,5.466335e+00,1.815022e+02,-8.986096e+00,3.826357e+00,-9.940241e-01
std,7.884501e+13,3.994521e+07,5.428293e+05,6.924959e+05,4.050818e+00,1.050709e+02,2.441379e+01,1.096255e+02,7.786687e-02
min,5.601266e+12,1.325376e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.519017e+01,-1.800000e+02,-1.000000e+00
25%,6.260384e+13,1.411057e+09,1.012003e+05,2.116006e+05,2.100000e+00,9.070000e+01,-2.602216e+01,-8.806982e+01,-1.000000e+00
50%,1.184859e+14,1.447828e+09,4.568181e+05,6.368242e+05,5.500000e+00,1.812000e+02,-1.497248e+01,-1.744785e+00,-1.000000e+00
75%,1.980758e+14,1.466633e+09,9.611991e+05,1.210012e+06,8.500000e+00,2.711000e+02,4.522833e+00,1.010775e+02,-1.000000e+00
max,2.812058e+14,1.480032e+09,4.430996e+06,7.181037e+06,1.023000e+02,5.110000e+02,8.333266e+01,1.799938e+02,7.500000e-01


In [16]:
# check for null values 
drifting_longlines_df.isnull().sum()

mmsi                    0
timestamp               0
distance_from_shore     0
distance_from_port      0
speed                  98
course                 98
lat                     0
lon                     0
is_fishing              0
source                  0
dtype: int64

In [32]:
# reformat unix timestamps into datetime
drifting_long_lines_formatted_timestamps = pd.to_datetime(drifting_longlines_df['timestamp'], unit = 's')
drifting_longlines_df.insert(2, 'timestamp_reformat', drifting_long_lines_formatted_timestamps)
drifting_longlines_df.head() 

,mmsi,timestamp,timestamp_reformat,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
330,1.263956e+13,1.338658e+09,2012-06-02 17:19:50,111123.328125,147593.812500,8.2,162.800003,18.608902,-17.205967,1.0,dalhousie_longliner
423,1.263956e+13,1.340094e+09,2012-06-19 08:12:54,329078.531250,362363.843750,5.0,333.899994,18.839836,-19.453779,1.0,dalhousie_longliner
570,1.263956e+13,1.345706e+09,2012-08-23 07:06:20,86831.046875,196301.062500,0.7,260.100006,19.276720,-17.323196,1.0,dalhousie_longliner
638,1.263956e+13,1.346391e+09,2012-08-31 05:24:34,98881.343750,166465.140625,4.2,43.400002,18.852455,-17.255033,1.0,dalhousie_longliner
677,1.263956e+13,1.346825e+09,2012-09-05 06:11:05,74247.757812,177480.859375,7.0,333.500000,19.101482,-17.147205,1.0,dalhousie_longliner


In [33]:
# add separate month, day of the week, year, and hour columns derived from timstamps
drifting_longlines_df.insert(3, 'year', drifting_longlines_df['timestamp_reformat'].dt.year)
# round to closest hour 
drifting_longlines_df.insert(4, 'hour', drifting_longlines_df['timestamp_reformat'].dt.round('H').dt.hour)
drifting_longlines_df.insert(5, 'day_of_week', drifting_longlines_df['timestamp_reformat'].dt.day_name())
drifting_longlines_df.insert(6, 'month', drifting_longlines_df['timestamp_reformat'].dt.month_name())

drifting_longlines_df.head()

,mmsi,timestamp,timestamp_reformat,year,hour,day_of_week,month,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
330,1.263956e+13,1.338658e+09,2012-06-02 17:19:50,2012,17,Saturday,June,111123.328125,147593.812500,8.2,162.800003,18.608902,-17.205967,1.0,dalhousie_longliner
423,1.263956e+13,1.340094e+09,2012-06-19 08:12:54,2012,8,Tuesday,June,329078.531250,362363.843750,5.0,333.899994,18.839836,-19.453779,1.0,dalhousie_longliner
570,1.263956e+13,1.345706e+09,2012-08-23 07:06:20,2012,7,Thursday,August,86831.046875,196301.062500,0.7,260.100006,19.276720,-17.323196,1.0,dalhousie_longliner
638,1.263956e+13,1.346391e+09,2012-08-31 05:24:34,2012,5,Friday,August,98881.343750,166465.140625,4.2,43.400002,18.852455,-17.255033,1.0,dalhousie_longliner
677,1.263956e+13,1.346825e+09,2012-09-05 06:11:05,2012,6,Wednesday,September,74247.757812,177480.859375,7.0,333.500000,19.101482,-17.147205,1.0,dalhousie_longliner


In [34]:
# count the unique mmsi's 
len(pd.unique(drifting_longlines_df['mmsi']))

110

## Fixed Gear 

In [20]:
# read in the data from the fixed gear vessels file 
fixed_gear_df = pd.read_csv(fixed_gear_file)

# peak at data 
fixed_gear_df.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,7.572519e+12,1.347664e+09,0.0,36054.625000,0.0,0.000000,42.798748,-8.944992,-1.0,gfw
1,7.572519e+12,1.348056e+09,0.0,36054.625000,0.0,0.000000,42.798717,-8.945075,-1.0,gfw
2,7.572519e+12,1.350409e+09,0.0,90970.296875,0.0,198.199997,43.106419,-9.215466,-1.0,gfw
3,7.572519e+12,1.350410e+09,0.0,90970.296875,0.0,186.899994,43.106434,-9.215431,-1.0,gfw
4,7.572519e+12,1.350411e+09,0.0,90970.296875,0.0,190.500000,43.106430,-9.215442,-1.0,gfw


In [21]:
# check data characteristics
fixed_gear_df.describe()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing
count,1.559137e+06,1.559137e+06,1.559137e+06,1.559137e+06,1.559137e+06,1.559137e+06,1.559137e+06,1.559137e+06,1.559137e+06
mean,1.530752e+14,1.421486e+09,3.761878e+04,5.989848e+04,2.227195e+00,1.877938e+02,5.095086e+01,1.274018e+00,-9.659100e-01
std,8.976383e+13,3.782830e+07,1.090188e+05,1.269729e+05,3.412790e+00,1.177506e+02,5.894565e+00,8.512244e+00,2.173124e-01
min,7.572519e+12,1.325625e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.326460e+01,-1.792441e+02,-1.000000e+00
25%,8.878018e+13,1.387594e+09,0.000000e+00,5.656715e+03,0.000000e+00,7.740000e+01,4.745566e+01,-3.909275e+00,-1.000000e+00
50%,1.305289e+14,1.427254e+09,0.000000e+00,2.690659e+04,1.000000e-01,2.055000e+02,5.050242e+01,-2.333808e+00,-1.000000e+00
75%,2.616830e+14,1.455255e+09,3.413126e+04,5.514391e+04,3.800000e+00,2.870000e+02,5.602126e+01,8.220293e+00,-1.000000e+00
max,2.802913e+14,1.480032e+09,3.099833e+06,1.181676e+07,1.023000e+02,5.110000e+02,8.479108e+01,1.709277e+02,1.000000e+00


In [23]:
# data shape
fixed_gear_df.shape

(1559137, 10)

In [24]:
# drop rows without data for is_fishing 
fixed_gear_df = fixed_gear_df[fixed_gear_df['is_fishing'] != -1]
fixed_gear_df.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
26358,7.572519e+12,1.420185e+09,38012.222656,103307.679688,1.8,29.000000,43.416580,-9.328747,0.0,gfw
26359,7.572519e+12,1.420185e+09,38909.199219,103389.921875,1.0,13.900000,43.418369,-9.326941,0.0,gfw
26360,7.572519e+12,1.420186e+09,38909.199219,103389.921875,0.8,28.500000,43.420197,-9.325145,0.0,gfw
26361,7.572519e+12,1.420187e+09,38909.199219,103389.921875,1.3,46.500000,43.421734,-9.323038,0.0,gfw
26362,7.572519e+12,1.420188e+09,38909.199219,103389.921875,0.7,217.300003,43.420063,-9.331503,0.0,gfw


In [25]:
# recheck data shape 
fixed_gear_df.shape

(41858, 10)

In [26]:
# check label distribution 
fixed_gear_df['is_fishing'].value_counts()

0.000000    29790
1.000000    10665
0.666667      538
0.250000      423
0.333333      405
0.800000       29
0.750000        6
0.400000        2
Name: is_fishing, dtype: int64

In [35]:
# check for null values 
fixed_gear_df.isnull().sum()

mmsi                   0
timestamp              0
distance_from_shore    0
distance_from_port     0
speed                  0
course                 0
lat                    0
lon                    0
is_fishing             0
source                 0
dtype: int64

In [37]:
# reformat unix timestamps into datetime 
fixed_gear_formatted_timestamps = pd.to_datetime(fixed_gear_df['timestamp'], unit = 's')
fixed_gear_df.insert(2, 'timestamp_reformat', fixed_gear_formatted_timestamps)
fixed_gear_df.head()

ValueError: cannot insert timestamp_reformat, already exists

In [38]:
# add separate month, day of the week, year, and hour columns derived from timstamps
fixed_gear_df.insert(3, 'year', fixed_gear_df['timestamp_reformat'].dt.year)
# round to closest hour
fixed_gear_df.insert(4, 'hour', fixed_gear_df['timestamp_reformat'].dt.round('H').dt.hour)
fixed_gear_df.insert(5, 'day_of_week', fixed_gear_df['timestamp_reformat'].dt.day_name())
fixed_gear_df.insert(6, 'month', fixed_gear_df['timestamp_reformat'].dt.month_name())

fixed_gear_df.head()

,mmsi,timestamp,timestamp_reformat,year,hour,day_of_week,month,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
26358,7.572519e+12,1.420185e+09,2015-01-02 07:44:07,2015,8,Friday,January,38012.222656,103307.679688,1.8,29.000000,43.416580,-9.328747,0.0,gfw
26359,7.572519e+12,1.420185e+09,2015-01-02 07:56:08,2015,8,Friday,January,38909.199219,103389.921875,1.0,13.900000,43.418369,-9.326941,0.0,gfw
26360,7.572519e+12,1.420186e+09,2015-01-02 08:08:09,2015,8,Friday,January,38909.199219,103389.921875,0.8,28.500000,43.420197,-9.325145,0.0,gfw
26361,7.572519e+12,1.420187e+09,2015-01-02 08:23:08,2015,8,Friday,January,38909.199219,103389.921875,1.3,46.500000,43.421734,-9.323038,0.0,gfw
26362,7.572519e+12,1.420188e+09,2015-01-02 08:38:07,2015,9,Friday,January,38909.199219,103389.921875,0.7,217.300003,43.420063,-9.331503,0.0,gfw


In [39]:
# count the unique mmsi's 
len(pd.unique(fixed_gear_df['mmsi']))

36

## Pole and Lines 

In [40]:
# read in data from the pole and lines vessel file
pole_and_lines_df = pd.read_csv(pole_and_line_file)

# peak at the data 
pole_and_lines_df.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.848346e+13,1.340882e+09,0.000000,2236.013184,0.0,0.000000,28.967354,-13.537797,-1.0,gfw
1,1.848346e+13,1.340884e+09,0.000000,2236.013184,0.0,125.199997,28.967373,-13.537838,-1.0,gfw
2,1.848346e+13,1.340885e+09,0.000000,2236.013184,0.0,0.000000,28.967354,-13.537838,-1.0,gfw
3,1.848346e+13,1.340888e+09,0.000000,2236.013184,0.0,0.000000,28.967354,-13.537838,-1.0,gfw
4,1.848346e+13,1.340925e+09,1999.950928,2828.357666,8.7,203.100006,28.929653,-13.543955,-1.0,gfw


In [41]:
# check data characteristics 
pole_and_lines_df.describe()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing
count,1.613150e+05,1.613150e+05,1.613150e+05,1.613150e+05,161315.000000,161315.000000,161315.000000,161315.000000,161315.000000
mean,7.659841e+13,1.414174e+09,4.894434e+04,7.383040e+04,2.111584,132.245911,33.456753,-4.570087,-0.967918
std,5.740577e+13,4.300647e+07,2.036499e+05,2.333604e+05,3.696588,117.424918,5.665633,53.524248,0.205442
min,1.848346e+13,1.327882e+09,0.000000e+00,0.000000e+00,0.000000,0.000000,-41.853848,-70.921013,-1.000000
25%,1.848346e+13,1.368384e+09,0.000000e+00,2.236013e+03,0.000000,3.300000,28.965940,-28.527719,-1.000000
50%,8.703142e+13,1.423536e+09,0.000000e+00,1.442185e+04,0.000000,115.199997,33.029419,-16.734444,-1.000000
75%,8.703142e+13,1.456109e+09,2.280295e+04,5.324341e+04,1.800000,228.699997,38.531128,-13.539565,-1.000000
max,2.145727e+14,1.480031e+09,2.110362e+06,3.005100e+06,102.300003,360.000000,77.078987,177.632980,1.000000


In [42]:
# data shape 
pole_and_lines_df.shape

(161315, 10)

In [43]:
# drop rows without data for is_fishing 
pole_and_lines_df = pole_and_lines_df[pole_and_lines_df['is_fishing'] != -1]
pole_and_lines_df.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
46903,1.848346e+13,1.427865e+09,0.0,2236.013184,0.0,137.899994,28.965490,-13.539565,0.0,gfw
46904,1.848346e+13,1.427866e+09,0.0,2236.013184,0.0,144.800003,28.965481,-13.539565,0.0,gfw
46905,1.848346e+13,1.427867e+09,0.0,2236.013184,0.0,0.000000,28.965487,-13.539558,0.0,gfw
46906,1.848346e+13,1.427867e+09,0.0,2236.013184,0.0,112.000000,28.965469,-13.539557,0.0,gfw
46907,1.848346e+13,1.427868e+09,0.0,2236.013184,0.0,95.300003,28.965473,-13.539560,0.0,gfw


In [44]:
# recheck data shape 
pole_and_lines_df.shape

(4250, 10)

In [48]:
# check label distribution 
pole_and_lines_df['is_fishing'].value_counts()

0.000000    3176
1.000000     833
0.333333     189
0.750000      31
0.166667      12
0.400000       7
0.666667       2
Name: is_fishing, dtype: int64